In [1]:
from libs import vgg16
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.framework.ops import reset_default_graph

In [ ]:
def show_img(content_og, style_og):
    fig, axs = plt.subplots(1, 2)
    axs[0].imshow(content_og)
    axs[0].set_title('Content Image')
    axs[0].grid('off')
    axs[1].imshow(style_og)
    axs[1].set_title('Style Image')
    axs[1].grid('off')
    plt.show()

In [ ]:
net = vgg16.get_vgg_model()
device = '/cpu:0'
#create a graph
g = tf.Graph()
with tf.Session(graph=g) as sess, g.device(device):
    tf.import_graph_def(net['graph_def'], name='net')
    names = [op.name for op in g.get_operations()]

In [ ]:
content_og = plt.imread('pic/farm.jpg')
style_og = plt.imread('pic/yell.jpg')
show_img(content_og, style_og )

In [ ]:
content_img0 = vgg16.preprocess(content_og)
content_img = content_img0[np.newaxis]
style_img0 = vgg16.preprocess(style_og)
style_img = style_img0[np.newaxis]

In [ ]:
#grab a placeholder for the input and output of the network
x = g.get_tensor_by_name(names[0] + ":0")
softmax = g.get_tensor_by_name(names[-2] + ":0")

In [ ]:
with tf.Session(graph=g) as sess, g.device(device):
    content_layer = 'net/conv4_2/conv4_2:0'
    content_features = g.get_tensor_by_name(content_layer).eval(
            session=sess,
            feed_dict={x: content_img,
                'net/dropout_1/random_uniform:0': [[1.0] * 4096],
                'net/dropout/random_uniform:0': [[1.0] * 4096]
            })
print(content_features.shape)

In [ ]:
style_layers = ['net/conv1_1/conv1_1:0',
                'net/conv2_1/conv2_1:0',
                'net/conv3_1/conv3_1:0',
                'net/conv4_1/conv4_1:0',
                'net/conv5_1/conv5_1:0']
style_activations = []

with tf.Session(graph=g) as sess, g.device(device):
    for style_i in style_layers:
        style_activation_i = g.get_tensor_by_name(style_i).eval(
            feed_dict={
                x: style_img,
                'net/dropout_1/random_uniform:0': [[1.0] * 4096],
                'net/dropout/random_uniform:0': [[1.0] * 4096]})
        style_activations.append(style_activation_i)

In [ ]:
style_features = []
for style_activation_i in style_activations:
    s_i = np.reshape(style_activation_i, [-1, style_activation_i.shape[-1]])
    gram_matrix = np.matmul(s_i.T, s_i) / s_i.size
    style_features.append(gram_matrix.astype(np.float32))

## Remapping

In [ ]:
reset_default_graph()
g = tf.Graph()
vgg = vgg16.get_vgg_model()

In [ ]:
with tf.Session(graph=g) as sess, g.device('/cpu:0'):
    net_input = tf.Variable(content_img)
    tf.import_graph_def(
        net['graph_def'],
        name='net',
        input_map={'images:0': net_input})

In [ ]:
with tf.Session(graph=g) as sess, g.device('/cpu:0'):
    content_loss = tf.nn.l2_loss((g.get_tensor_by_name(content_layer) -
                                 content_features) /
                                 content_features.size)

In [ ]:
with tf.Session(graph=g) as sess, g.device('/cpu:0'):
    style_loss = np.float32(0.0)
    for style_layer_i, style_gram_i in zip(style_layers, style_features):
        layer_i = g.get_tensor_by_name(style_layer_i)
        layer_shape = layer_i.get_shape().as_list()
        layer_size = layer_shape[1] * layer_shape[2] * layer_shape[3]
        layer_flat = tf.reshape(layer_i, [-1, layer_shape[3]])
        gram_matrix = tf.matmul(tf.transpose(layer_flat), layer_flat) / layer_size
        style_loss = tf.add(style_loss, tf.nn.l2_loss((gram_matrix - style_gram_i) / np.float32(style_gram_i.size)))

In [ ]:
def total_variation_loss(x):
    h, w = x.get_shape().as_list()[1], x.get_shape().as_list()[1]
    dx = tf.square(x[:, :h-1, :w-1, :] - x[:, :h-1, 1:, :])
    dy = tf.square(x[:, :h-1, :w-1, :] - x[:, 1:, :w-1, :])
    return tf.reduce_sum(tf.pow(dx + dy, 1.25))

with tf.Session(graph=g) as sess, g.device('/cpu:0'):
    tv_loss = total_variation_loss(net_input)

## Training

In [ ]:
with tf.Session(graph=g) as sess, g.device('/cpu:0'):
    loss = 0.1 * content_loss + 5.0 * style_loss + 0.01 * tv_loss
    optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

In [ ]:
with tf.Session(graph=g) as sess, g.device('/cpu:0'):
    sess.run(tf.global_variables_initializer())
    # map input to noise
    n_iterations = 60
    og_img = net_input.eval()
    imgs = []
    for it_i in range(n_iterations):
        _, this_loss, synth = sess.run([optimizer, loss, net_input],
                feed_dict={
                    'net/dropout_1/random_uniform:0':
                        np.ones(g.get_tensor_by_name(
                        'net/dropout_1/random_uniform:0').get_shape().as_list()),
                    'net/dropout/random_uniform:0':
                        np.ones(g.get_tensor_by_name(
                        'net/dropout/random_uniform:0').get_shape().as_list())})
        if it_i % 50 == 0:
            print("%d: %f, (%f - %f)" %
            (it_i, this_loss, np.min(synth), np.max(synth)))
            imgs.append(np.clip(synth[0], 0, 1))
            fig, ax = plt.subplots(1, 3, figsize=(22, 10))
            ax[0].imshow(vgg16.deprocess(content_img0))
            ax[0].set_title('content image')
            ax[1].imshow(vgg16.deprocess(style_img0))
            ax[1].set_title('style image')
            ax[2].set_title('current synthesis')
            ax[2].imshow(vgg16.deprocess(synth[0]))
            plt.show()
            fig.canvas.draw()
    gif.build_gif(imgs, saveto='stylenet-bosch.gif')